In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kushagratandon12/diabetic-retinopathy-balanced

Dataset URL: https://www.kaggle.com/datasets/kushagratandon12/diabetic-retinopathy-balanced
License(s): CC0-1.0
100% 1.89G/1.90G [00:46<00:00, 47.0MB/s]
100% 1.90G/1.90G [00:46<00:00, 43.9MB/s]


In [ ]:
!unzip /content/diabetic-retinopathy-balanced.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30457_left.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30474_right.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30496_left._aug_21.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30496_right.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30509_right._aug_24.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30509_right.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30567_right.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30597_left._aug_28.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30597_left.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/3059_left.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/3070_right._aug_3.jpeg  
  inflating: /content/content/Diabetic_Balanced_Data/val/2/30717_ri

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
data_dir = "/content/content/Diabetic_Balanced_Data"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.efficientnet_b3(pretrained=False)
"""we have switched off pretrained because we dont want to include pretrained weights of efficientB3 we have our own weights on our saved model pytorch file"""


num_classes = 5
model.classifier[1] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

model = model.to(device)
model.load_state_dict(torch.load('efficientModel.pt'), strict = False)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-6-21c701174934>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

_IncompatibleKeys(missing_keys=['classifier.1.1.weight', 'classifier.1.1.bias'], unexpected_keys=['classifier.1.weight', 'classifier.1.bias'])

In [ ]:
"""Just do simple data augmentation techniques i have not applied data augmentation technique which increase samples of individual classes will do it later Insha'Allah"""
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(300),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(330),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [ ]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)


train_loader = DataLoader(train_dataset ,batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset ,batch_size=16, shuffle=False)

In [ ]:

"""To minimize overfitting we apply L2 Regularization which is basically a constraint applied on model :)"""
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)


scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)



In [ ]:
num_epochs = 20
curr_acc = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # print('here 1')

    if epoch == 3:
        for param in model.features[-5:].parameters():
            param.requires_grad = True

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # print('here 2')


    scheduler.step()


    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, LR: {scheduler.get_last_lr()[0]}')


    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    if val_accuracy > curr_acc:
      torch.save(model.state_dict(), 'fineTunedEfficientnet_b3.pt')
      curr_acc = val_accuracy
    print(f'Validation Accuracy: {val_accuracy:.2f}%')


Epoch 1/20, Loss: 0.9883, LR: 0.0001
Validation Accuracy: 76.26%
Epoch 2/20, Loss: 0.8621, LR: 0.0001
Validation Accuracy: 77.29%
Epoch 3/20, Loss: 0.8150, LR: 0.0001
Validation Accuracy: 77.33%
Epoch 4/20, Loss: 0.7855, LR: 0.0001
Validation Accuracy: 78.29%
Epoch 5/20, Loss: 0.7474, LR: 1e-05
Validation Accuracy: 78.76%
Epoch 6/20, Loss: 0.7211, LR: 1e-05
Validation Accuracy: 79.35%
Epoch 7/20, Loss: 0.7091, LR: 1e-05
Validation Accuracy: 79.35%
Epoch 8/20, Loss: 0.6993, LR: 1e-05
Validation Accuracy: 79.49%
Epoch 9/20, Loss: 0.6910, LR: 1e-05
Validation Accuracy: 79.75%
Epoch 10/20, Loss: 0.6886, LR: 1.0000000000000002e-06
Validation Accuracy: 79.68%
Epoch 11/20, Loss: 0.6876, LR: 1.0000000000000002e-06
Validation Accuracy: 79.73%
Epoch 12/20, Loss: 0.6876, LR: 1.0000000000000002e-06
Validation Accuracy: 79.64%
Epoch 13/20, Loss: 0.6897, LR: 1.0000000000000002e-06
Validation Accuracy: 79.39%
Epoch 14/20, Loss: 0.6905, LR: 1.0000000000000002e-06
Validation Accuracy: 79.71%
